## Introduction

Tree methods are one of the most efficient ways of handling both the classification and the regression problems and there are ample of methods available to choose from like **Decision Tree, Random forest and Gradient Boosting**. In this article we will use the **official [dataset](https://github.com/apache/spark/blob/master/data/mllib/sample_libsvm_data.txt) provided by Spark** (perfectly cleaned and ready to use) so that we won't be focussing much on data preprocessing and more on the model development and evaluating process.

By this way we can go through each tree algorithm in a detailed and descriptive way. So without wasting more time in story building let's get our hands on building **tree model using PySpark's MLIB**.

## Installing PySpark

Before using the PySpark's methods, libraries and utilities one will have to install the pyspark which is quite a straightforward step, we just have to use a simple command i.e. **pip install pyspark**. 

**Note:** In the below cell note that I used **"!"** before command which denotes that it is the **jupyter notebook** cell if one is using the command line then exclamation sign is not required.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 31.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=11a067bbe7ee644552220a7b079f167c8fdf7bc197025e04547080e0f9b491ba
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


## Importing required PySpark libraries

Let's start by **importing the libraries as per the initial requirements** we might end up import more libraries as we move on with more functionalities meanwhile let's get started with **first round of modules**.

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

So we have imported mainly three libraries for now and they are Pipeline, RandomForestClassifier and MulticlassClassificationEvaluator let's get a brief introduction for all three of them.

1. **RandomForestClassifier:** As name denotes it is the **Random forest algorithm for the classification problems**.

2. **Pipeline:** This module helps us to maintain the proper workflow of the machine learning process and **staging each step** for hassle free load balancing.

3. **MulticlassClassificationEvaluator:** This is the model evaulation metric more specifically for the **multi class classification**.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('random_forest_intro').getOrCreate()
spark

**Inference:** In the above set of codes we imported the **SparkSession module** and then we have created the **Spark object** which is kind of the "**must-todo**" step for accessing and utilising all the **PySpark's methods, libraries and modules.**

In [4]:
data_tree = spark.read.format("libsvm").load("sample_libsvm_data.txt")

**Inference:** So now we are loading that official dataset from the spark github repository. Note that this one is **not** in the traditional **CSV** format instead in **libsvm** format hence we will load it in that way only. 

In [5]:
data_tree.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



**Inference:** Here is the sneek peak of the dataset on which we will be working to **apply tree algorithms** one need to notice that it is already preprocessed and cleaned just the way PySpark want it to be and infact we don't even have to use **VectorAssembler** object because that task is also done.

In [6]:
data_tree.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



**Inference:** As discussed earlier that we have preprocessed data hence one is the features column **(Vector type)** and the label column i.e. **target column**

In [7]:
data_tree.head(5)

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

**Inference:** There is one more way to have a look into the dataset and this one is quite similar to pandas i.e. the **head()** method which not only will return the **name and type of the columns** but also the **values** each are holding.

## Train-Test split

This phase of the machine learning cycle is also known as **splitting the dataset** phase where we will breakdown the dataset into training and testing set so that we can **train the model** on **training set** and **test** the same on **testing set**.

In [8]:
(trainingSet, testSet) = data_tree.randomSplit([0.7, 0.3])

**Inference:** **RandomSplit()** is the method which is responsible to divide the dataset into training and testing set and from the parameter values we can stimulate that there is **70% of training data and 30% of testing data**.

Now we will **train** the **random forest model (here classifier)** for that firstly the random forest object needs to be created by **passing in the relevant parameter**.

In [9]:
random_forest = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20)

**Inference:** While creating the Random forest classifer we are passing the **label** column that is our target and the **features** column (collectively all the features) and as it is **random forest classifier** so we have to specify the **total number of trees** as **20**

In [10]:
model_rf = random_forest.fit(trainingSet)

**Inference:** Now to actually train the model we use the **fit()** method by passing the set of training data as the parameter to that function and don't forget to use the random forest object for calling it.

In [11]:
predictions = model_rf.transform(testSet)

**Inference:** Here comes the stage where we will make **predictions** using the **evaluate** method of **MLIB** library and make sure to evaluate the model on testing data as then only the purpose of using it will be fullfilled.

In [12]:
predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



**Inference:** Let's look what the prediction DataFrame holds. So from the above output we can see that there are **5** columns:
1. **Label:** This is the target column.
2. **features:** All the **features/dependent** columns in the form of **vector**
3. **rawPrediction:** This column will be very handy in the case of **GBT classifier** 
4. **Probability:** It holds the **probability** of how much is the chnces that the predcitions is correct.
5. **Predictions:** **Predicted** values by model during evaluation.

In [13]:
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[95,96,97,12...|
|       0.0|  0.0|(692,[98,99,100,1...|
|       0.0|  0.0|(692,[121,122,123...|
|       0.0|  0.0|(692,[122,123,124...|
|       0.0|  0.0|(692,[123,124,125...|
+----------+-----+--------------------+
only showing top 5 rows



**Inference:** In the above output we have filtered the main DataFrame to extract the important columns only i.e. **prediction, label and features.**

In [14]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

**Inference:** Here we are in the stage of model evaluation and for that we are using the Multi class classification Evaluator there is one key difference between **Binary** and **Multi class evaluators**, binary on one side can only return the **AUC** curve while other one can return the **accuracy, precision and recall metrics** as well.

In [15]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0


**Inference:** Well!! to get the **0 test error** seems to be bit unacceptable especially in the real setup, having said that keep one thing in mind that this dataset is **highly seperable and cleaned** so one might expect such tremendeous results.

## Gradient Boosted Trees

**Gradient boosting Trees (GBT's)** is another tree method which can be used for both **classification** and **regression** problems though GBT's are built on the basis of **ensemble methods** using number of **decision trees**. Though one don't have to worry about the mathematics behind this algorithm as Spark handles it in a better way.


In [16]:
from pyspark.ml.classification import GBTClassifier


data_gbt = spark.read.format("libsvm").load("sample_libsvm_data.txt")


(trainingSet, testSet) = data_gbt.randomSplit([0.7, 0.3])

gbt_mdl = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

model = gbt_mdl.fit(trainingSet)

# Make predictions.
predictions = model.transform(testSet)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[95,96,97,12...|
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
+----------+-----+--------------------+
only showing top 5 rows



**Code breakdown:** This is just for the walkthrough purpose otherwise if you have learnt the random forest part then this one will be easy to pick up in terms of implementation.


1. Firstly importing the **GBT model** and reading the dataset by exactly the same method.
2. Splitting the dataset into **training** and **testing** set using **randomSplit** method.
3. Training of the GBT model using object creation and **fit** method.
4. Making **predictions** using the **transform** method on testing data.

In [17]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0625


**Inference:** As we did in the case of random forest similarly following the same approach for **GBT** where we got the test result as around **0.088** which seems bit realistic.


**Note:** The same pipeline can be applied in the case of decision tree as well.

## Conclusion

In this article we have discussed about all the important **tree methods** that can be implemented using **PySpark's MLIB** and went through the hands on practice by using the official documented dataset provided by **Spark**. Now, let's discuss everything we did in a nutshell.

1. First we did an **environment setup thing** and reading the dataset and then look at the dataset which was preprocessed and ready to use for model development.

2. Then the stage of **splitting the dataset** comes into existence following by the **random forest** model development, prediction and in the end evaluation.

3. Similarly we perform the same process in the case of **GBT model** (practically) and conclude that using any tree method need to follow the same process.